In [1]:
pip install gcastle

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['CASTLE_BACKEND'] = 'pytorch'

from collections import OrderedDict

import networkx as nx

import castle
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import IIDSimulation, DAG
from castle.algorithms import PC, GES, ICALiNGAM, GOLEM

import matplotlib.pyplot as plt

2023-05-02 10:11:26,473 - /home/matthew/anaconda3/lib/python3.9/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [5]:
#Finds correlation between variables
file_path_1 = 'E.Bidar_AVR_TAVI_2017_2022_versturen.csv'
df_1 = pd.read_csv(file_path_1)
file_path_2 = 'MYOFRAIL_HU_data.csv'
df_2 = pd.read_csv(file_path_2)
df_1 = df_1.rename(columns = {"pat_nr": "Patient_number"})
df = pd.concat([df_1, df_2])

matrix = df.corr(
    method = 'pearson',  # The method of correlation
    min_periods = 1      # Min number of observations required
)
# Set a correlation threshold
corr_threshold = 0.3

# Create a mask to only show highly correlated values (excluding the diagonal)
mask = np.zeros_like(matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True
mask[np.abs(matrix) <= corr_threshold] = True

# Create the heatmap
#sns.heatmap(matrix, annot=True, mask=mask, cmap='coolwarm', vmin=-1, vmax=1, center=0, linewidths=.5, square=True)
#plt.show()

highly_corr_pairs = np.where(np.abs(matrix) > corr_threshold)
highly_corr_pairs = [(matrix.index[x], matrix.columns[y]) for x, y in zip(*highly_corr_pairs) if x != y and x < y]
print("Highly correlated pairs:", highly_corr_pairs)
highly_corr_matrix = matrix[(np.abs(matrix) > corr_threshold) & ~(np.eye(matrix.shape[0], dtype=bool))]
highly_corr_matrix = highly_corr_matrix.dropna(axis=0, how='all').dropna(axis=1, how='all')
print (highly_corr_matrix)

corr_matrix = matrix
highly_corr_indices = np.where(np.abs(corr_matrix) > corr_threshold)
highly_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*highly_corr_indices) if x != y and x < y]

# Get the unique column names of highly correlated pairs
highly_corr_columns = list(set([col for pair in highly_corr_pairs for col in pair]))

# Create a new DataFrame with only highly correlated columns
df_highly_corr = df[highly_corr_columns]

# Display the new DataFrame
print(df_highly_corr)

print(len(df_1))

Highly correlated pairs: [('overleden_zh', 'overleden_locatie'), ('overleden_zh', 'CVA_restletsel'), ('overleden_zh', 'rethorac_30d'), ('overleden_zh', 'herfixatie_30d'), ('overleden_zh', 'DSWI_30d'), ('overleden_zh', 'mort_status'), ('overleden_locatie', 'CVA_restletsel'), ('overleden_locatie', 'rethorac_30d'), ('overleden_locatie', 'herfixatie_30d'), ('overleden_locatie', 'DSWI_30d'), ('overleden_locatie', 'mort_status'), ('nieuwe_interv', 'arm_beenwond'), ('nieuwe_interv', 'beademing'), ('nieuwe_interv', 'rethorac_30d'), ('nieuwe_interv', 're_aklep'), ('periop_MI', 're_MI'), ('arm_beenwond', 'beademing'), ('arm_beenwond', 'heropname_IC'), ('arm_beenwond', 'nierfalen'), ('arm_beenwond', 're_aklep'), ('urineweginfectie', 'beademing'), ('urineweginfectie', 'heropname_IC'), ('urineweginfectie', 'nierfalen'), ('urineweginfectie', 'IC duur'), ('beademing', 'heropname_IC'), ('beademing', 'nierfalen'), ('beademing', 'IC duur'), ('heropname_IC', 'nierfalen'), ('CVA_restletsel', 'herfixatie_3